# **Análise:** 'Escola Nota 10': como é a educação na cidade do CE onde mais cresceu número de unidades premiadas?

Notebook com análise de dados para produção de notícia para o *Diário do Nordeste* sobre o Escola Nota 10 é uma premiação criada por lei, da Secretaria da Educação do Ceará (Seduc), que reconhece as unidades de ensino que apresentam os melhores desempenhos dos alunos nas avaliações do Sistema Permanente de Avaliação da Educação Básica do Ceará (Spaece).

Encontramos que a cidade de Massapê (CE) foi a que mais teve uma variação positiva no número de escolas reconhecidas na premiação entre as edições realizadas em 2019 e 2022. A escola de ensino fundamental Padre Linhares, especificamente, foi premiada nas três séries avaliadas: 2º ano (alfabetização), 5º ano e 9º ano (matemática e português).

A partir desses indicadores, buscamos a secretaria municipal da educação de Massapê, assim como a direção e a coordenação da escola Padre Linhares, para entender as medidas adotadas no município e na unidade de ensino para conseguir esses bons resultados.

A matéria foi publicada no dia 26 de junho de 2023 e pode ser lida [neste link, no Diário do Nordeste](https://diariodonordeste.verdesmares.com.br/ceara/escola-nota-10-como-e-a-educacao-na-cidade-do-ce-onde-mais-cresceu-numero-de-unidades-premiadas-1.3384890/).

**Coleta e análise de dados:** Gabriela Custódio

**Reportagem:** Gabriela Custódio

**Edição da reportagem:** Dahiana Araújo

In [17]:
import pandas as pd
import numpy as np
import re

In [98]:
path = 'arquivos_utilizados/'

In [101]:
df = pd.read_csv(path + 'Escolas Nota 10 - Total.csv')

In [102]:
arquivos_gerados = 'arquivos_gerados/'

In [103]:
the_geom = df[['Geometria', 'Código', 'Município']]

In [104]:
data = df[df['Premiada?'] == 'SIM']

## Ceará

In [105]:
# quantidade de prêmios por ano
premios_municipios_ano = data[['Ano', 'Código', 'Município']].value_counts().reset_index()
premios_municipios_ano_pivot = premios_municipios_ano.pivot_table(index=['Código', 'Município'], columns='Ano', values=0).fillna(0).reset_index()
premios_municipios_ano_pivot['Mudança_2022_2019'] = premios_municipios_ano_pivot[2022].sub(premios_municipios_ano_pivot[2019], axis=0)
premios_municipios_ano_pivot = premios_municipios_ano_pivot.sort_values('Mudança_2022_2019', ascending=False)

In [106]:
# quantidade de escolas premiadas
escolas_premiadas_municipios = data.drop_duplicates(subset=['Escola', 'Ano'])
escolas_premiadas_municipios = escolas_premiadas_municipios[['Ano', 'Município']].value_counts().reset_index()
escolas_premiadas_municipios_pivot = escolas_premiadas_municipios.pivot_table(index='Município', columns='Ano', values=0).fillna(0).reset_index()
escolas_premiadas_municipios_pivot['Mudança_2022_2019'] = escolas_premiadas_municipios_pivot[2022].sub(escolas_premiadas_municipios_pivot[2019])
escolas_premiadas_municipios_pivot = escolas_premiadas_municipios_pivot.sort_values('Mudança_2022_2019', ascending=False)
escolas_premiadas_municipios_pivot.head(2)

Ano,Município,2019,2022,Mudança_2022_2019
58,MASSAPÊ,3.0,18.0,15.0
86,SOBRAL,18.0,28.0,10.0


In [107]:
escolas_premiadas_municipios_pivot.to_csv(arquivos_gerados + 'escolas_premiadas_municipios_pivot.csv', index=False)

In [108]:
escolas_premiadas_municipios_pivot[escolas_premiadas_municipios_pivot[2022] != 0]['Município'].nunique()

76

In [109]:
escolas_premiadas_municipios_2022 = escolas_premiadas_municipios_pivot[escolas_premiadas_municipios_pivot[2022] != 0]
escolas_premiadas_municipios_2022[escolas_premiadas_municipios_2022['Mudança_2022_2019'] < 0]['Município'].nunique()

27

### CONFERIR NOMES DAS ESCOLAS

In [110]:
premios_por_escola_e_ano = data[['Ano', 'Município', 'Escola']].value_counts().reset_index()

In [111]:
# para ver a lista de escola completa, colocar 580
pd.set_option('display.max_rows', 580)

In [112]:
premios_por_escola_e_ano = premios_por_escola_e_ano.pivot_table(index=['Município', 'Escola'], columns='Ano', values=0, aggfunc='sum').reset_index().fillna(0).sort_values(by=['Município', 'Escola'])

In [113]:
def multiple_replace(string, rep_dict):
    pattern = re.compile("|".join([re.escape(k) for k in sorted(rep_dict,key=len,reverse=True)]), flags=re.DOTALL)
    return pattern.sub(lambda x: rep_dict[x.group(0)], string)

In [114]:
from unicodedata import normalize

def remover_acentos(txt):
    return normalize('NFKD', txt).encode('ASCII', 'ignore').decode('ASCII')

In [115]:
lista = []
for i in premios_por_escola_e_ano['Escola']:
    #  aplicar função para remover acentuação
    i = remover_acentos(i)
    
    # aplicar função para fazer diversas substituições
    
    teste = multiple_replace(i, {'EEFI': '', 'ETI': '', 'EMEB': '', 'EMEIF': '', 'EEIEF': '', 'EEIF': '', 'EEF': '', 'EIEF': '', 'E CRECHE': '',
                               'E E F': '', 'CEIF': '', 'ESCOLA DE ENSINO BASICO': '', 'EEB': '', ' - ': '', 'E E I F': '',
                               'EMEF': '', 'C E I F': '', 'EM TEMPO INTEGRAL': '', 'ESCOLA': '', 'ESC': '', 'EMEIEF': '', 'CMEF': '',
                               'ESC MUL ENS FUND': '', 'COL MUL ENS FUND': '', 'CENTRO EDUCACIONAL EM TEMPO\nINTEGRAL': '',
                               'MUNICIPAL DE ENSINO FUNDAMENTAL': '', 'MA DO CARMO CARNEIRO': 'PROFESSORA MARIA DO CARMO CARNEIRO',
                               'DE ENSINO FUNDAMENTAL': '', 'MUNICIPAL DE ENSINO INFANTIL E\nFUNDAMENTAL': '', 'MUNICIPAL DE ENSINO': ''})
    lista.append(teste)

In [116]:
premios_por_escola_e_ano['Escola_Novo'] = pd.DataFrame(lista)

In [117]:
# remover espaços antes e depois da string

premios_por_escola_e_ano['Escola_Novo'] = premios_por_escola_e_ano['Escola_Novo'].str.strip()

In [118]:
premios_por_escola_e_ano[premios_por_escola_e_ano['Município'] == 'MASSAPÊ'].sort_values('Escola_Novo', ascending=True)

Ano,Município,Escola,2019,2022,Escola_Novo
247,MASSAPÊ,ANTONIO DE PAULA PESSOA FIGUEIREDO EEF,0.0,2.0,ANTONIO DE PAULA PESSOA FIGUEIREDO
248,MASSAPÊ,AURELIANO LOPES EEF,0.0,2.0,AURELIANO LOPES
249,MASSAPÊ,BRAS SATIRO NASCIMENTO EEF,0.0,2.0,BRAS SATIRO NASCIMENTO
250,MASSAPÊ,CENTRO COMUNITARIO FRANCISCO CANUTO EEF,0.0,2.0,CENTRO COMUNITARIO FRANCISCO CANUTO
252,MASSAPÊ,EEF CENTRO COMUNITARIO SÃO JOÃO BATISTA,1.0,1.0,CENTRO COMUNITARIO SAO JOAO BATISTA
251,MASSAPÊ,CENTRO COMUNITARIO SAO JOSE EEF,0.0,2.0,CENTRO COMUNITARIO SAO JOSE
253,MASSAPÊ,EEF CORIOLANO GOMES FROTA,1.0,0.0,CORIOLANO GOMES FROTA
254,MASSAPÊ,EEF DEPUTADO FRANCISCO ALMEIDA MONTE,1.0,1.0,DEPUTADO FRANCISCO ALMEIDA MONTE
255,MASSAPÊ,FRANCISCA DIOGO GOMES EEF,0.0,1.0,FRANCISCA DIOGO GOMES
256,MASSAPÊ,FRANCISCO CARDOSO FERREIRA LIMA EEF,0.0,2.0,FRANCISCO CARDOSO FERREIRA LIMA


In [119]:
premios_por_escola_e_ano[[2022, 'Município', 'Escola_Novo']].groupby(['Município']).sum().reset_index().sort_values(by=[2022], ascending=False)

Ano,Município,2022
86,SOBRAL,40.0
58,MASSAPÊ,26.0
36,GUARACIABA DO NORTE,18.0
79,QUIXERAMOBIM,15.0
72,PEDRA BRANCA,14.0
88,SÃO BENEDITO,14.0
68,NOVO ORIENTE,11.0
78,QUITERIANÓPOLIS,11.0
80,RERIUTABA,9.0
24,CRUZ,9.0


## Massapê

In [120]:
massape = data[data['Município'] == 'MASSAPÊ']

In [121]:
massape['Escola_Title'] = massape['Escola'].str.title()

C:\Users\55859\AppData\Local\Temp/ipykernel_764/316693787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  massape['Escola_Title'] = massape['Escola'].str.title()


In [122]:
# quantidade de prêmios por ano
massape['Ano'].value_counts()

2022    26
2019     3
Name: Ano, dtype: int64

In [123]:
# quantidade de prêmios por ano
premios_municipios_ano_massape = massape[['Ano', 'Município']].value_counts().reset_index()
premios_municipios_ano_massape_pivot = premios_municipios_ano_massape.pivot_table(index='Município', columns='Ano', values=0).fillna(0).reset_index()
premios_municipios_ano_massape_pivot['Mudança_2022_2019'] = premios_municipios_ano_massape_pivot[2022].sub(premios_municipios_ano_massape_pivot[2019], axis=0)
premios_municipios_ano_massape_pivot = premios_municipios_ano_massape_pivot.sort_values('Mudança_2022_2019', ascending=False)
premios_municipios_ano_massape_pivot

Ano,Município,2019,2022,Mudança_2022_2019
0,MASSAPÊ,3,26,23


In [124]:
# premiações por ano e série

premios_massape = massape[['Município', 'Escola', 'Série',	'Ano']][['Ano', 'Série']].value_counts().reset_index().rename(columns={0: 'Quantidade de prêmios'})

In [125]:
premios_massape.to_csv(arquivos_gerados + 'premios_massape_serie_ano.csv', index=False)

In [126]:
premios_massape_pivot = premios_massape.pivot_table(index='Série', columns='Ano').reset_index().fillna(0)

In [127]:
premios_massape_pivot.to_csv(arquivos_gerados + 'premios_massape_serie_ano_pivot.csv', index=False)

In [128]:
massape_escolas = massape[['Escola', 'Ano']].drop_duplicates()

In [129]:
# quantidade de escolas premiadas por ano
massape_escolas['Ano'].value_counts()

2022    18
2019     3
Name: Ano, dtype: int64

In [130]:
# quantidade de escolas premiadas
escolas_premiadas_municipios_massape = massape.drop_duplicates(subset=['Escola', 'Ano'])
escolas_premiadas_municipios_massape = escolas_premiadas_municipios_massape[['Ano', 'Município']].value_counts().reset_index()
escolas_premiadas_municipios_massape_pivot = escolas_premiadas_municipios_massape.pivot_table(index='Município', columns='Ano', values=0).fillna(0).reset_index()
escolas_premiadas_municipios_massape_pivot['Mudança_2022_2019'] = escolas_premiadas_municipios_massape_pivot[2022].sub(escolas_premiadas_municipios_massape_pivot[2019])
escolas_premiadas_municipios_massape_pivot = escolas_premiadas_municipios_massape_pivot.sort_values('Mudança_2022_2019', ascending=False)
escolas_premiadas_municipios_massape_pivot

Ano,Município,2019,2022,Mudança_2022_2019
0,MASSAPÊ,3,18,15


In [131]:
# premiações por escola e ano

premios_massape_escola_ano = massape_escolas[['Ano']].value_counts().reset_index().rename(columns={0: 'Quantidade de escolas premiadas'})

In [132]:
premios_massape_escola_ano.to_csv(arquivos_gerados + 'premios_massape_escola_ano.csv', index=False)

In [133]:
massape[massape['Escola'].str.contains('PADRE LINHARES EEF')]

,Ord.,Município,Escola,Série,Ano,Premiada?,Código,Geometria,Escola_Title
921,122,MASSAPÊ,PADRE LINHARES EEF,2º Ano,2022,SIM,2308005,"{""type"":""MultiPolygon"",""coordinates"":[[[[-40.3...",Padre Linhares Eef
1185,16,MASSAPÊ,PADRE LINHARES EEF,5º Ano,2022,SIM,2308005,"{""type"":""MultiPolygon"",""coordinates"":[[[[-40.3...",Padre Linhares Eef
1312,16,MASSAPÊ,PADRE LINHARES EEF,9º Ano,2022,SIM,2308005,"{""type"":""MultiPolygon"",""coordinates"":[[[[-40.3...",Padre Linhares Eef


In [134]:
escolas_premiadas_ano = massape[['Escola', 'Série', 'Ano', 'Código']].groupby(['Escola', 'Série', 'Ano']).count().reset_index().pivot_table(index=['Escola'], columns=['Ano', 'Série']).fillna(0).reset_index()

In [135]:
escolas_premiadas_ano.to_csv('escolas_premiadas_ano.csv', index=False)

In [136]:
escolas_premiadas_massape_2022 = massape[massape['Ano'] == 2022]

In [137]:
massape[['Ano', 'Série', 'Escola']][massape['Ano'] == 2022].groupby(['Escola', 'Série']).count()

Ano
Escola                                   Série      
ANTONIO DE PAULA PESSOA FIGUEIREDO EEF   5º Ano    1
                                         9º Ano    1
AURELIANO LOPES EEF                      2º Ano    1
                                         5º Ano    1
BRAS SATIRO NASCIMENTO EEF               2º Ano    1
                                         5º Ano    1
CENTRO COMUNITARIO FRANCISCO CANUTO EEF  2º Ano    1
                                         5º Ano    1
CENTRO COMUNITARIO SAO JOSE EEF          2º Ano    1
                                         5º Ano    1
EEF CENTRO COMUNITARIO SÃO JOÃO BATISTA  2º Ano    1
EEF DEPUTADO FRANCISCO ALMEIDA MONTE     5º Ano    1
FRANCISCA DIOGO GOMES EEF                5º Ano    1
FRANCISCO CARDOSO FERREIRA LIMA EEF      2º Ano    1
                                         5º Ano    1
GERARDO EMILIANO EEF                     9º Ano    1
JOSE ESTEVAO VASCONCELOS EEF             9º Ano    1
MA DO CARMO CARNEIRO EEF                 9º Ano    1
MA LAURA SOARES FROTA EEF                9º Ano    1
MONSENHOR MANUEL HENRIQUES DE ARAUJO EEF 5º Ano    1
PADRE LINHARES EEF                       2º Ano    1
                                         5º Ano    1
                                         9º Ano    1
PEDRO RUFINO FURTADO EEF                 5º Ano    1
PROFESSORA MARIA DO CARMO CARNEIRO CMEF  9º Ano    1
SANTO ANTONIO EEF                        2º Ano    1

## Censo da Educação Básica

In [138]:
pd.set_option('display.max_columns', 30)

In [139]:
censo = pd.read_csv(path + 'microdados_ed_basica_2022.csv', encoding='latin-1', sep=';')

C:\Users\55859\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (27) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [140]:
censo_massape = censo[censo['NO_MUNICIPIO'] == 'Massapê']

In [141]:
# selecionando as colunas de interesse

educacao_basica_massape = censo_massape[['SG_UF', 'CO_UF', 'NO_MUNICIPIO', 'CO_MUNICIPIO', 'NO_ENTIDADE', 'CO_ENTIDADE', 'TP_DEPENDENCIA',
                                         'TP_LOCALIZACAO', 'TP_LOCALIZACAO_DIFERENCIADA', 'TP_SITUACAO_FUNCIONAMENTO', 'IN_FUND', 'QT_MAT_FUND',
                                         'QT_MAT_FUND_INT', 'QT_MAT_BAS_FEM', 'QT_MAT_BAS_MASC', 'QT_MAT_BAS_ND', 'QT_MAT_BAS_BRANCA', 'QT_MAT_BAS_PRETA',
                                         'QT_MAT_BAS_PARDA', 'QT_MAT_BAS_AMARELA', 'QT_MAT_BAS_INDIGENA', 'QT_DOC_FUND', 'QT_TUR_FUND']]

In [142]:
# filtrando pelas escolas que oferecem turmas da educação básica

educacao_basica_massape_ens_fund = educacao_basica_massape[educacao_basica_massape['IN_FUND'] == 1]

In [143]:
# checando se todas as escolas estão em funcionamento ('TP_SITUACAO_FUNCIONAMENTO' == 1)

educacao_basica_massape_ens_fund['TP_SITUACAO_FUNCIONAMENTO'].unique()

array([1], dtype=int64)

In [144]:
# checando a que esfera pertence cada escola

dependencia = {
    1: 'Federal',
    2: 'Estadual',
    3: 'Municipal',
    4: 'Privada'
}

dependencia_escolas_ens_fund_massape = educacao_basica_massape_ens_fund[['CO_ENTIDADE', 'TP_DEPENDENCIA']].value_counts().reset_index()
dependencia_escolas_ens_fund_massape['TP_DEPENDENCIA'] = dependencia_escolas_ens_fund_massape['TP_DEPENDENCIA'].replace(dependencia)
dependencia_escolas_ens_fund_massape.head()

,CO_ENTIDADE,TP_DEPENDENCIA,0
0,23020423,Municipal,1
1,23021152,Privada,1
2,23272686,Municipal,1
3,23240946,Municipal,1
4,23237090,Privada,1


In [145]:
# checando quantas escolas pertencem a cada esfera

dependencia_escolas_ens_fund_massape.groupby('TP_DEPENDENCIA').sum().reset_index().rename(columns={0: 'QUANTIDADE_ESCOLAS'})

,TP_DEPENDENCIA,CO_ENTIDADE,QUANTIDADE_ESCOLAS
0,Municipal,599642216,26
1,Privada,115790262,5


In [146]:
# checando a que localização pertence cada escola

localizacao = {
    1: 'Urbana',
    2: 'Rural'
}

localizacao_escolas_ens_fund_massape = educacao_basica_massape_ens_fund[['CO_ENTIDADE', 'TP_LOCALIZACAO']].value_counts().reset_index()
localizacao_escolas_ens_fund_massape['TP_LOCALIZACAO'] = localizacao_escolas_ens_fund_massape['TP_LOCALIZACAO'].replace(localizacao)
localizacao_escolas_ens_fund_massape.head()

,CO_ENTIDADE,TP_LOCALIZACAO,0
0,23020423,Rural,1
1,23021152,Urbana,1
2,23272686,Urbana,1
3,23240946,Urbana,1
4,23237090,Urbana,1


In [147]:
# checando quantas escolas pertencem a cada localização

localizacao_escolas_ens_fund_massape[['TP_LOCALIZACAO', 0]].groupby('TP_LOCALIZACAO').sum().reset_index().rename(columns={0: 'QUANTIDADE_ESCOLAS'})

,TP_LOCALIZACAO,QUANTIDADE_ESCOLAS
0,Rural,9
1,Urbana,22


In [148]:
# checando a que localização diferenciada pertence cada escola

localizacao_diferenciada = {
    0: 'A escola não está em área de localização diferenciada',
    1: 'Área de assentamento',
    2: 'Terra indígena',
    3: 'Área onde se localiza comunidade remanescente de quilombos'
}

localizacao_diferenciada_escolas_ens_fund_massape = educacao_basica_massape_ens_fund[['CO_ENTIDADE', 'TP_LOCALIZACAO_DIFERENCIADA']].value_counts().reset_index()
localizacao_diferenciada_escolas_ens_fund_massape['TP_LOCALIZACAO_DIFERENCIADA'] = localizacao_diferenciada_escolas_ens_fund_massape['TP_LOCALIZACAO_DIFERENCIADA'].replace(localizacao_diferenciada)
localizacao_diferenciada_escolas_ens_fund_massape.head()

,CO_ENTIDADE,TP_LOCALIZACAO_DIFERENCIADA,0
0,23020423,A escola não está em área de localização difer...,1
1,23021152,A escola não está em área de localização difer...,1
2,23272686,A escola não está em área de localização difer...,1
3,23240946,A escola não está em área de localização difer...,1
4,23237090,A escola não está em área de localização difer...,1


In [149]:
# checando quantas escolas pertencem a cada localização diferenciada

localizacao_diferenciada_escolas_ens_fund_massape[['TP_LOCALIZACAO_DIFERENCIADA', 0]].groupby('TP_LOCALIZACAO_DIFERENCIADA').sum().reset_index().rename(columns={0: 'QUANTIDADE_ESCOLAS'})

,TP_LOCALIZACAO_DIFERENCIADA,QUANTIDADE_ESCOLAS
0,A escola não está em área de localização difer...,31


In [150]:
# verificando a quantidade de matrículas na educação básica do município

qtd_matriculas = educacao_basica_massape_ens_fund[['NO_ENTIDADE', 'CO_ENTIDADE', 'QT_MAT_FUND']]
qtd_matriculas['QT_MAT_FUND'].sum()

5000.0

In [151]:
# verificando matrículas por gênero

genero_alunos = educacao_basica_massape_ens_fund[['NO_ENTIDADE', 'CO_ENTIDADE', 'QT_MAT_BAS_FEM', 'QT_MAT_BAS_MASC']]
genero_alunos[['QT_MAT_BAS_FEM', 'QT_MAT_BAS_MASC']].sum()

QT_MAT_BAS_FEM     3074.0
QT_MAT_BAS_MASC    3218.0
dtype: float64

In [152]:
# verificando matrículas por cor/raça

cor_raca_alunos = educacao_basica_massape_ens_fund[['NO_ENTIDADE', 'CO_ENTIDADE', 'QT_MAT_BAS_ND', 'QT_MAT_BAS_BRANCA',
                                           'QT_MAT_BAS_PRETA', 'QT_MAT_BAS_PARDA', 'QT_MAT_BAS_AMARELA', 'QT_MAT_BAS_INDIGENA']]

cor_raca_alunos[['QT_MAT_BAS_ND', 'QT_MAT_BAS_BRANCA','QT_MAT_BAS_PRETA', 'QT_MAT_BAS_PARDA', 'QT_MAT_BAS_AMARELA', 'QT_MAT_BAS_INDIGENA']].sum().reset_index()

,index,0
0,QT_MAT_BAS_ND,488.0
1,QT_MAT_BAS_BRANCA,396.0
2,QT_MAT_BAS_PRETA,53.0
3,QT_MAT_BAS_PARDA,5340.0
4,QT_MAT_BAS_AMARELA,11.0
5,QT_MAT_BAS_INDIGENA,4.0


In [153]:
# verificando a quantidade de docentes na educação básica de massapê

docentes_ens_fund_massape = educacao_basica_massape_ens_fund[['NO_ENTIDADE', 'CO_ENTIDADE', 'QT_DOC_FUND']]
docentes_ens_fund_massape[['QT_DOC_FUND']].sum()

QT_DOC_FUND    333.0
dtype: float64

In [154]:
# verificando a quantidade de turmas na educação básica de massapê

turmas_ens_fund_massape = educacao_basica_massape_ens_fund[['NO_ENTIDADE', 'CO_ENTIDADE', 'QT_TUR_FUND']]
turmas_ens_fund_massape[['QT_TUR_FUND']].sum()

QT_TUR_FUND    231.0
dtype: float64

In [155]:
# selecionar as escolas nota 10

escolas_nota_dez = ['SANTO ANTONIO EEF', 'CENTRO COMUNITARIO FRANCISCO CANUTO EEF', 'CENTRO COMUNITARIO SAO JOAO BATISTA EEF', 'FRANCISCO CARDOSO FERREIRA LIMA EEF',
                    'BRAS SATIRO NASCIMENTO EEF', 'AURELIANO LOPES EEF', 'CENTRO COMUNITARIO SAO JOSE EEF', 'PADRE LINHARES EEF', 'FRANCISCA DIOGO GOMES EEF',
                    'ANTONIO DE PAULA PESSOA FIGUEIREDO EEF', 'PEDRO RUFINO FURTADO EEF', 'DEPUTADO FRANCISCO ALMEIDA MONTE EEF', 'MONSENHOR MANUEL HENRIQUES DE ARAUJO EEF',
                    'GERARDO EMILIANO EEF', 'MA DO CARMO CARNEIRO EEF', 'MA LAURA SOARES FROTA EEF', 'JOSE ESTEVAO VASCONCELOS EEF', 'PROFESSORA MARIA DO CARMO CARNEIRO CMEF']

educacao_basica_massape_ens_fund['PREMIACAO'] = educacao_basica_massape_ens_fund['NO_ENTIDADE'].apply(lambda x: 'Premiada' if x in escolas_nota_dez else 'Não Premiada')

C:\Users\55859\AppData\Local\Temp/ipykernel_764/3212610064.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  educacao_basica_massape_ens_fund['PREMIACAO'] = educacao_basica_massape_ens_fund['NO_ENTIDADE'].apply(lambda x: 'Premiada' if x in escolas_nota_dez else 'Não Premiada')


In [156]:
educacao_basica_massape_ens_fund['PREMIACAO'].value_counts()

Premiada        18
Não Premiada    13
Name: PREMIACAO, dtype: int64

#### Verificar informações sobre as escolas premiadas como Nota 10

In [157]:
escolas_premiadas_massape = educacao_basica_massape_ens_fund[educacao_basica_massape_ens_fund['PREMIACAO'] == 'Premiada']

In [158]:
escolas_premiadas_massape['NO_ENTIDADE_TITLE'] = escolas_premiadas_massape['NO_ENTIDADE'].str.title()

C:\Users\55859\AppData\Local\Temp/ipykernel_764/311466119.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  escolas_premiadas_massape['NO_ENTIDADE_TITLE'] = escolas_premiadas_massape['NO_ENTIDADE'].str.title()


In [159]:
# verificando a quantidade de matrículas nas escolas nota 10 de Massapê em 2022

qtd_matriculas_escolas_nota_10 = escolas_premiadas_massape[['NO_ENTIDADE', 'CO_ENTIDADE', 'QT_MAT_FUND']]
qtd_matriculas_escolas_nota_10['QT_MAT_FUND'].sum()

3400.0

In [160]:
# verificando matrículas por gênero nas escolas nota 10

qtd_matriculas_genero_alunos_escolas_nota_10 = escolas_premiadas_massape[['NO_ENTIDADE', 'CO_ENTIDADE', 'QT_MAT_BAS_FEM', 'QT_MAT_BAS_MASC']]
qtd_matriculas_genero_alunos_escolas_nota_10[['QT_MAT_BAS_FEM',	'QT_MAT_BAS_MASC']].sum()

QT_MAT_BAS_FEM     1993.0
QT_MAT_BAS_MASC    2130.0
dtype: float64

In [161]:
# verificando matrículas por cor/raça nas escolas nota 10

qtd_matriculas_cor_raca_alunos_escolas_nota_10 = escolas_premiadas_massape[['NO_ENTIDADE', 'CO_ENTIDADE', 'QT_MAT_BAS_ND', 'QT_MAT_BAS_BRANCA',
                                           'QT_MAT_BAS_PRETA', 'QT_MAT_BAS_PARDA', 'QT_MAT_BAS_AMARELA', 'QT_MAT_BAS_INDIGENA']]

qtd_matriculas_cor_raca_alunos_escolas_nota_10[['QT_MAT_BAS_ND', 'QT_MAT_BAS_BRANCA','QT_MAT_BAS_PRETA', 'QT_MAT_BAS_PARDA', 'QT_MAT_BAS_AMARELA', 'QT_MAT_BAS_INDIGENA']].sum()

QT_MAT_BAS_ND           352.0
QT_MAT_BAS_BRANCA       224.0
QT_MAT_BAS_PRETA         34.0
QT_MAT_BAS_PARDA       3503.0
QT_MAT_BAS_AMARELA        7.0
QT_MAT_BAS_INDIGENA       3.0
dtype: float64

In [162]:
# verificando a quantidade de docentes que lecionavam nas escolas nota 10 de massapê em 2022

docentes_edbasica_nota10_massape = escolas_premiadas_massape[['NO_ENTIDADE', 'CO_ENTIDADE', 'QT_DOC_FUND']]
docentes_edbasica_nota10_massape[['QT_DOC_FUND']].sum()

QT_DOC_FUND    219.0
dtype: float64

In [163]:
# verificando a quantidade de turmas nas escolas nota 10 de massapê em 2022

turmas_edbasica_escolas_nota10_massape = escolas_premiadas_massape[['NO_ENTIDADE', 'CO_ENTIDADE', 'QT_TUR_FUND']]
turmas_edbasica_escolas_nota10_massape[['QT_TUR_FUND']].sum()

QT_TUR_FUND    150.0
dtype: float64

#### Escola Padre Linhares

In [164]:
eef_padre_linhares = escolas_premiadas_massape[escolas_premiadas_massape['NO_ENTIDADE'].str.contains('PADRE LINHARES')]

In [165]:
eef_padre_linhares = eef_padre_linhares[['NO_ENTIDADE', 'NO_ENTIDADE_TITLE', 'QT_MAT_FUND', 'QT_MAT_FUND_INT',
                    'QT_MAT_BAS_FEM', 'QT_MAT_BAS_MASC', 'QT_MAT_BAS_ND',
                    'QT_MAT_BAS_BRANCA', 'QT_MAT_BAS_PRETA', 'QT_MAT_BAS_PARDA',
                    'QT_MAT_BAS_AMARELA', 'QT_MAT_BAS_INDIGENA', 'QT_DOC_FUND',
                    'QT_TUR_FUND', 'PREMIACAO', ]]

In [166]:
eef_padre_linhares.T

,53437
NO_ENTIDADE,PADRE LINHARES EEF
NO_ENTIDADE_TITLE,Padre Linhares Eef
QT_MAT_FUND,265.0
QT_MAT_FUND_INT,7.0
QT_MAT_BAS_FEM,128.0
QT_MAT_BAS_MASC,137.0
QT_MAT_BAS_ND,20.0
QT_MAT_BAS_BRANCA,14.0
QT_MAT_BAS_PRETA,1.0
QT_MAT_BAS_PARDA,230.0
